In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
HEIGHT = 112
WIDTH = 56

In [4]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from __future__ import division, print_function, unicode_literals
import tensorflow as tf
from time import time
from keras.utils import np_utils#one hot
import os
from skimage import data
from skimage import transform
import skimage
import numpy as np

# Download the dataset


def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                  if os.path.isdir(os.path.join(data_directory,d))]
    labels=[]
    images=[]
    for d in directories:
        label_directory = os.path.join(data_directory,d)
        file_names = [os.path.join(label_directory,f)
                     for f in os.listdir(label_directory)
                     if f.endswith('.jpg')]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(str(d))
    return images, labels

Using TensorFlow backend.


In [5]:
ROOT_PATH = '/home/tang/targa/exp3/'
train_data_directory = os.path.join(ROOT_PATH, "padding112_56/train")
#validation_data_directory = os.path.join(ROOT_PATH, "16_8_1568_train") 16_8_4448_train  16_8_1248_train 

test_data_directory = os.path.join(ROOT_PATH, "padding112_56/test")

In [6]:
train_images,train_labels = load_data(train_data_directory)
test_images,test_labels = load_data(test_data_directory)


In [7]:
#encoding trY teY
from numpy import array
#from numpy import argmax
#from keras.utils import to_categorical
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example

#train_labels = np_utils.to_categorical(train_labels,num_classes = None)
#test_labels = np_utils.to_categorical(test_labels,num_classes = None)
trL,teL=array(train_labels),array(test_labels)
label_encoder = LabelEncoder()

In [8]:
integer_encoded_trL = label_encoder.fit_transform(trL)
integer_encoded_teL = label_encoder.fit_transform(teL)


In [9]:
trainLabels = np_utils.to_categorical(integer_encoded_trL,num_classes = 32)
testLabels = np_utils.to_categorical(integer_encoded_teL,num_classes = 32)


In [10]:
########convert rgb images into black and white
#convert images into size of vector
from skimage.color import rgb2gray

trainImages = rgb2gray(np.array(train_images))
testImages = rgb2gray(np.array(test_images))

train_flat_images = np.reshape(trainImages,(len(trainImages),HEIGHT*WIDTH))
test_flat_images = np.reshape(testImages,(len(testImages),HEIGHT*WIDTH))


In [11]:
train_flat_images[0].shape

(6272,)

In [12]:
print('train images shape:\t\t{}'.format(train_flat_images.shape))

print('test images shape:\t\t{}'.format(test_flat_images.shape))



train images shape:		(1246, 6272)
test images shape:		(411, 6272)


In [13]:
batch_size = 100
n_batch = len(trainLabels)//batch_size
lr = tf.Variable(0.001,dtype = tf.float32)

In [14]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histgram',var) # 直方图

In [15]:

#命名空间
with tf.name_scope('input'):
    #定义两个placeholder 
    #把批次的图片传进去，开始是100张一次的传送，直到最后
    x = tf.placeholder(tf.float32,[None,WIDTH*HEIGHT],name='x-input')
    y = tf.placeholder(tf.float32,[None,32],name='y-input')
    #定义一个dropout的值
    keep_prob = tf.placeholder(tf.float32,name = 'non-dropout-rate')
with tf.name_scope('layer'):
    with tf.name_scope('W1'):
        #第一层
        W1 = tf.Variable(tf.truncated_normal([WIDTH*HEIGHT,50],stddev=0.1),name='W1')#改成截断正太分布
    with tf.name_scope('b1'):
        
        b1 = tf.Variable(tf.zeros([50])+0.1,name = 'b1')
    with tf.name_scope('out1'):
        
        output1 = tf.matmul(x,W1) + b1
    with tf.name_scope('L1'):
        L1 = tf.nn.tanh(output1)
    with tf.name_scope('L1-drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L1_drop = tf.nn.dropout(L1,keep_prob)

    '''
    with tf.name_scope('W2'):
        #第二层
        W2 = tf.Variable(tf.truncated_normal([1000,2000],stddev=0.1),name='W2')
    with tf.name_scope('b2'):
        b2 = tf.Variable(tf.zeros([2000])+0.2,name='b2')
    with tf.name_scope('output2'):
        output2 = tf.matmul(L1_drop,W2)+b2
    with tf.name_scope('L2'):
        
        L2 = tf.nn.tanh(output2)
    with tf.name_scope('L2_drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L2_drop = tf.nn.dropout(L2,keep_prob)

    with tf.name_scope('W3'):
        #第三层
        W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1),name='W3')
    with tf.name_scope('b3'):
        b3 = tf.Variable(tf.zeros([1000])+0.1,name='b3')
    with tf.name_scope('output3'):   
        output3 = tf.matmul(L2_drop,W3)+b3
    with tf.name_scope('L3'):
        L3 = tf.nn.tanh(output3)
    with tf.name_scope('L3-drop'):
        #L1为第一层的输出，keep_prob为1的话就是不丢失，如果为0的话就是全丢，如果为0.5的话就是丢失50%的神经元
        L3_drop = tf.nn.dropout(L3,keep_prob)
        '''
    with tf.name_scope('W4'):
    #第四层
        W4 = tf.Variable(tf.truncated_normal([50,32],stddev=0.1))
    with tf.name_scope('b4'):        
        b4 = tf.Variable(tf.zeros([32])+0.1)
    with tf.name_scope('output4'):
        output4 = tf.matmul(L1_drop,W4)+ b4
    with tf.name_scope('softmax'):
        #预测分类
        prediction = tf.nn.softmax(output4)
with tf.name_scope('loss'):
    #定义一个损失函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits = prediction))#联合交叉熵
    tf.summary.scalar('cross_entropy',loss)
    #loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('train'):
    #定义一个方法来收敛神经网络，使用梯度下降法,学习率=0.1
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    #train_step = tf.train.AdamOptimizer(0.1).minimize(loss)
#初始化
init = tf.global_variables_initializer()
with tf.name_scope('accuracy'):
    with tf.name_scope('correct-prediction'):
        #定义一个求准确率的方法
        #如果有两个相同的就会返回TRUE，不相同就返回false，然后就依次存入correct_prediction
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中的最大值
    with tf.name_scope('accuracy'):    
        #求准确率
        #首先把bool值转化成32位的浮点数，然后求平均值
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [16]:
merged = tf.summary.merge_all()

In [17]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [18]:
with tf.Session() as sess:
    sess.run(init)
    #路径和文件名
    train_writer = tf.summary.FileWriter('logs/padding112*56targa/train',sess.graph)
   
    test_writer = tf.summary.FileWriter('logs/padding112*56targa/test',sess.graph)
    for epoch in range(200001):#把所有的图片训练20次
        #一批一批的迭代图片，一次迭代100张，一共运行n_batch次
        for batch in range(n_batch):
            #每次传入一百张图片
            batch_xs,batch_ys = next_batch(batch_size,train_flat_images,trainLabels)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        train_batch_xs,train_batch_ys = train_flat_images,trainLabels
        summary = sess.run(merged,feed_dict={x:train_batch_xs,y:train_batch_ys,keep_prob:1.0})           
        train_writer.add_summary(summary,epoch)
            
        test_batch_xs,test_batch_ys = test_flat_images,testLabels
        summary = sess.run(merged,feed_dict={x:test_batch_xs,y:test_batch_ys,keep_prob:1.0})
        test_writer.add_summary(summary,epoch)  
        
        test_acc = sess.run(accuracy,feed_dict={x:test_flat_images,y:testLabels,keep_prob:1.0})        
        train_acc = sess.run(accuracy,feed_dict={x:train_flat_images,y:trainLabels,keep_prob:1.0})
        test_loss = sess.run(loss,feed_dict={x:test_flat_images,y:testLabels,keep_prob:1.0}) 
        train_loss = sess.run(loss,feed_dict={x:train_flat_images,y:trainLabels,keep_prob:1.0})
        if epoch%2000 == 0:
            
            print('epoch=' + str(epoch) + ',train accuracy='+ str(train_acc)+',test accuracy=' + str(test_acc)+',train loss='+str(train_loss)+',test loss='+str(test_loss))

epoch=0,train accuracy=0.050561797,test accuracy=0.03892944,train loss=3.4652512,test loss=3.4651346
epoch=2000,train accuracy=0.63804173,test accuracy=0.40389293,train loss=2.8989818,test loss=3.1219194
epoch=4000,train accuracy=0.65569824,test accuracy=0.42092457,train loss=2.8678484,test loss=3.1015253
epoch=6000,train accuracy=0.6621188,test accuracy=0.43065694,train loss=2.857281,test loss=3.0909414
epoch=8000,train accuracy=0.66853935,test accuracy=0.43309003,train loss=2.8499498,test loss=3.0894775
epoch=10000,train accuracy=0.6693419,test accuracy=0.42579076,train loss=2.847845,test loss=3.091828
epoch=12000,train accuracy=0.6693419,test accuracy=0.42335767,train loss=2.8472002,test loss=3.0906842
epoch=14000,train accuracy=0.670947,test accuracy=0.43065694,train loss=2.845068,test loss=3.0864027
epoch=16000,train accuracy=0.6741573,test accuracy=0.42579076,train loss=2.8421507,test loss=3.0864654
epoch=18000,train accuracy=0.676565,test accuracy=0.42092457,train loss=2.839489,

epoch=160000,train accuracy=0.75842696,test accuracy=0.50121653,train loss=2.757498,test loss=3.0172837
epoch=162000,train accuracy=0.75842696,test accuracy=0.4914842,train loss=2.7574577,test loss=3.0187356
epoch=164000,train accuracy=0.75842696,test accuracy=0.4939173,train loss=2.7574549,test loss=3.0195062
epoch=166000,train accuracy=0.75842696,test accuracy=0.4914842,train loss=2.75745,test loss=3.0206478
epoch=168000,train accuracy=0.75842696,test accuracy=0.4939173,train loss=2.7574086,test loss=3.020586
epoch=170000,train accuracy=0.75922954,test accuracy=0.4914842,train loss=2.7568893,test loss=3.0218918
epoch=172000,train accuracy=0.75922954,test accuracy=0.4939173,train loss=2.7566922,test loss=3.0213716
epoch=174000,train accuracy=0.75922954,test accuracy=0.4914842,train loss=2.7566764,test loss=3.0230608
epoch=176000,train accuracy=0.75922954,test accuracy=0.4914842,train loss=2.7566698,test loss=3.0224247
epoch=178000,train accuracy=0.75922954,test accuracy=0.4939173,trai

In [ ]:
epoch=0,train accuracy=0.018459069,test accuracy=0.024330901,train loss=3.4654872,test loss=3.465808
epoch=200,train accuracy=0.16372392,test accuracy=0.10948905,train loss=3.4072504,test loss=3.427533
epoch=400,train accuracy=0.36757624,test accuracy=0.23844282,train loss=3.2421467,test loss=3.320766
epoch=600,train accuracy=0.5104334,test accuracy=0.30413625,train loss=3.062754,test loss=3.21667
epoch=800,train accuracy=0.5497592,test accuracy=0.32360098,train loss=3.0047889,test loss=3.1919427
epoch=1000,train accuracy=0.58426964,test accuracy=0.35036495,train loss=2.9666758,test loss=3.1688979
epoch=1200,train accuracy=0.5955056,test accuracy=0.3673966,train loss=2.9500375,test loss=3.1583383
epoch=1400,train accuracy=0.5987159,test accuracy=0.37226278,train loss=2.939597,test loss=3.1504176
epoch=1600,train accuracy=0.60192615,test accuracy=0.37226278,train loss=2.9321404,test loss=3.1456485
epoch=1800,train accuracy=0.60674155,test accuracy=0.37956205,train loss=2.9258687,test loss=3.1400473
epoch=2000,train accuracy=0.60995185,test accuracy=0.38199514,train loss=2.9206815,test loss=3.1364515
epoch=2200,train accuracy=0.6123595,test accuracy=0.38199514,train loss=2.9168122,test loss=3.1344275
epoch=2400,train accuracy=0.61476725,test accuracy=0.38686132,train loss=2.9129305,test loss=3.131798
epoch=2600,train accuracy=0.61556983,test accuracy=0.38686132,train loss=2.9101646,test loss=3.1314323
epoch=2800,train accuracy=0.6179775,test accuracy=0.38929442,train loss=2.9079022,test loss=3.1301823
epoch=3000,train accuracy=0.6179775,test accuracy=0.38442823,train loss=2.9060576,test loss=3.12996
epoch=3200,train accuracy=0.61958265,test accuracy=0.38686132,train loss=2.90423,test loss=3.1291854
epoch=3400,train accuracy=0.6203852,test accuracy=0.3941606,train loss=2.902931,test loss=3.1289513
epoch=3600,train accuracy=0.64044946,test accuracy=0.40389293,train loss=2.8846753,test loss=3.1181757
epoch=3800,train accuracy=0.64205456,test accuracy=0.40632603,train loss=2.8814695,test loss=3.1143892
epoch=4000,train accuracy=0.6444623,test accuracy=0.40875912,train loss=2.8785024,test loss=3.1136067
epoch=4200,train accuracy=0.6444623,test accuracy=0.40632603,train loss=2.87731,test loss=3.1143835
epoch=4400,train accuracy=0.64526486,test accuracy=0.40875912,train loss=2.8763216,test loss=3.1128592
epoch=4600,train accuracy=0.64686996,test accuracy=0.40632603,train loss=2.875318,test loss=3.1124985
epoch=4800,train accuracy=0.64767253,test accuracy=0.40389293,train loss=2.8744245,test loss=3.111874
epoch=5000,train accuracy=0.64767253,test accuracy=0.40632603,train loss=2.873742,test loss=3.1120462
epoch=5200,train accuracy=0.64686996,test accuracy=0.40875912,train loss=2.8732538,test loss=3.1117933
epoch=5400,train accuracy=0.64686996,test accuracy=0.40875912,train loss=2.8728094,test loss=3.112349
epoch=5600,train accuracy=0.64767253,test accuracy=0.40875912,train loss=2.87242,test loss=3.112764
epoch=5800,train accuracy=0.64767253,test accuracy=0.40632603,train loss=2.8718042,test loss=3.1127214
epoch=6000,train accuracy=0.6492777,test accuracy=0.40632603,train loss=2.8708498,test loss=3.1122754
epoch=6200,train accuracy=0.6492777,test accuracy=0.40632603,train loss=2.8701956,test loss=3.1118238
epoch=6400,train accuracy=0.65008026,test accuracy=0.40875912,train loss=2.8695207,test loss=3.1113067
epoch=6600,train accuracy=0.65088284,test accuracy=0.40875912,train loss=2.8686044,test loss=3.1104455
epoch=6800,train accuracy=0.65088284,test accuracy=0.4111922,train loss=2.8679569,test loss=3.1106186
epoch=7000,train accuracy=0.6516854,test accuracy=0.40875912,train loss=2.8673527,test loss=3.1112218
epoch=7200,train accuracy=0.65088284,test accuracy=0.40875912,train loss=2.8670628,test loss=3.1118393
epoch=7400,train accuracy=0.6516854,test accuracy=0.40875912,train loss=2.8667905,test loss=3.111094
epoch=7600,train accuracy=0.65088284,test accuracy=0.40875912,train loss=2.8666117,test loss=3.1100087
epoch=7800,train accuracy=0.65088284,test accuracy=0.40875912,train loss=2.866403,test loss=3.1110094
epoch=8000,train accuracy=0.65088284,test accuracy=0.40875912,train loss=2.86621,test loss=3.1116858
epoch=8200,train accuracy=0.6516854,test accuracy=0.40875912,train loss=2.865854,test loss=3.1111815
epoch=8400,train accuracy=0.65248793,test accuracy=0.40632603,train loss=2.8651268,test loss=3.111129
epoch=8600,train accuracy=0.6532905,test accuracy=0.40632603,train loss=2.8646255,test loss=3.110453
epoch=8800,train accuracy=0.6532905,test accuracy=0.40632603,train loss=2.8644977,test loss=3.110401
epoch=9000,train accuracy=0.6532905,test accuracy=0.40875912,train loss=2.8641644,test loss=3.1090086
epoch=9200,train accuracy=0.6532905,test accuracy=0.40632603,train loss=2.8639085,test loss=3.1093833
epoch=9400,train accuracy=0.6540931,test accuracy=0.40875912,train loss=2.8634884,test loss=3.1087916
epoch=9600,train accuracy=0.6540931,test accuracy=0.40632603,train loss=2.8631642,test loss=3.1101367
epoch=9800,train accuracy=0.65489566,test accuracy=0.40632603,train loss=2.8628414,test loss=3.1101303
epoch=10000,train accuracy=0.65489566,test accuracy=0.40875912,train loss=2.8623118,test loss=3.1106234
epoch=10200,train accuracy=0.65489566,test accuracy=0.40632603,train loss=2.8620138,test loss=3.1103556
epoch=10400,train accuracy=0.65489566,test accuracy=0.40632603,train loss=2.8615477,test loss=3.1077974
epoch=10600,train accuracy=0.6565008,test accuracy=0.40145984,train loss=2.8609273,test loss=3.10989
epoch=10800,train accuracy=0.6565008,test accuracy=0.40389293,train loss=2.8604808,test loss=3.1100314
epoch=11000,train accuracy=0.6573034,test accuracy=0.40389293,train loss=2.8601549,test loss=3.1088955
epoch=11200,train accuracy=0.6573034,test accuracy=0.40632603,train loss=2.8597796,test loss=3.1095912
epoch=11400,train accuracy=0.6573034,test accuracy=0.40389293,train loss=2.8593447,test loss=3.1098876
epoch=11600,train accuracy=0.6581059,test accuracy=0.40389293,train loss=2.8589785,test loss=3.1095066
epoch=11800,train accuracy=0.6581059,test accuracy=0.40389293,train loss=2.8587055,test loss=3.1100314
epoch=12000,train accuracy=0.6581059,test accuracy=0.40389293,train loss=2.8585308,test loss=3.1113129
epoch=12200,train accuracy=0.6581059,test accuracy=0.40389293,train loss=2.8580654,test loss=3.1100261
epoch=12400,train accuracy=0.65971106,test accuracy=0.40389293,train loss=2.8570998,test loss=3.1091623
epoch=12600,train accuracy=0.66051364,test accuracy=0.40145984,train loss=2.856346,test loss=3.1103826
epoch=12800,train accuracy=0.6613162,test accuracy=0.40389293,train loss=2.8557332,test loss=3.1091018
epoch=13000,train accuracy=0.6613162,test accuracy=0.40145984,train loss=2.855372,test loss=3.1111174
epoch=13200,train accuracy=0.6613162,test accuracy=0.40389293,train loss=2.8551106,test loss=3.1098497
epoch=13400,train accuracy=0.6621188,test accuracy=0.40145984,train loss=2.8546386,test loss=3.1098726
epoch=13600,train accuracy=0.6621188,test accuracy=0.40389293,train loss=2.8541946,test loss=3.1096916
epoch=13800,train accuracy=0.66292137,test accuracy=0.40632603,train loss=2.853627,test loss=3.1083055
epoch=14000,train accuracy=0.66292137,test accuracy=0.40632603,train loss=2.8534877,test loss=3.1080644
epoch=14200,train accuracy=0.66292137,test accuracy=0.40875912,train loss=2.853272,test loss=3.105997
epoch=14400,train accuracy=0.66292137,test accuracy=0.4136253,train loss=2.8530097,test loss=3.105276
epoch=14600,train accuracy=0.66372395,test accuracy=0.40875912,train loss=2.8526704,test loss=3.106136
epoch=14800,train accuracy=0.66372395,test accuracy=0.4111922,train loss=2.852197,test loss=3.1044924
epoch=15000,train accuracy=0.66452646,test accuracy=0.40632603,train loss=2.8516316,test loss=3.1045935
epoch=15200,train accuracy=0.66532904,test accuracy=0.4111922,train loss=2.851417,test loss=3.1038895
epoch=15400,train accuracy=0.66532904,test accuracy=0.40875912,train loss=2.8510048,test loss=3.1038585
epoch=15600,train accuracy=0.66532904,test accuracy=0.4136253,train loss=2.8508873,test loss=3.1029239
epoch=15800,train accuracy=0.66532904,test accuracy=0.4136253,train loss=2.8507593,test loss=3.1029596
epoch=16000,train accuracy=0.66532904,test accuracy=0.4136253,train loss=2.8506403,test loss=3.1020122
epoch=16200,train accuracy=0.6661316,test accuracy=0.4136253,train loss=2.8499792,test loss=3.1016693
epoch=16400,train accuracy=0.6669342,test accuracy=0.4136253,train loss=2.8494747,test loss=3.101662
epoch=16600,train accuracy=0.6669342,test accuracy=0.4136253,train loss=2.8493779,test loss=3.1014936
epoch=16800,train accuracy=0.6669342,test accuracy=0.4160584,train loss=2.8492196,test loss=3.1033707
epoch=17000,train accuracy=0.6669342,test accuracy=0.4160584,train loss=2.84911,test loss=3.1010444
epoch=17200,train accuracy=0.6669342,test accuracy=0.4136253,train loss=2.8490407,test loss=3.1011963
epoch=17400,train accuracy=0.6669342,test accuracy=0.4111922,train loss=2.848997,test loss=3.102856
epoch=17600,train accuracy=0.6669342,test accuracy=0.4160584,train loss=2.8488457,test loss=3.100793
epoch=17800,train accuracy=0.6669342,test accuracy=0.4136253,train loss=2.8485744,test loss=3.102251
epoch=18000,train accuracy=0.66773677,test accuracy=0.4160584,train loss=2.8482265,test loss=3.099734
epoch=18200,train accuracy=0.66773677,test accuracy=0.4136253,train loss=2.8479564,test loss=3.1010818
epoch=18400,train accuracy=0.66773677,test accuracy=0.4160584,train loss=2.8478827,test loss=3.0995579
epoch=18600,train accuracy=0.66773677,test accuracy=0.4160584,train loss=2.8474174,test loss=3.0994506
epoch=18800,train accuracy=0.6693419,test accuracy=0.41849148,train loss=2.8469484,test loss=3.0987477
epoch=19000,train accuracy=0.67014444,test accuracy=0.42579076,train loss=2.8462005,test loss=3.0943654
epoch=19200,train accuracy=0.670947,test accuracy=0.42822385,train loss=2.845565,test loss=3.0927641
epoch=19400,train accuracy=0.67014444,test accuracy=0.42822385,train loss=2.8454442,test loss=3.090425
epoch=19600,train accuracy=0.6717496,test accuracy=0.43552312,train loss=2.8447375,test loss=3.0870547
epoch=19800,train accuracy=0.6717496,test accuracy=0.43552312,train loss=2.8446457,test loss=3.0872395
epoch=20000,train accuracy=0.6717496,test accuracy=0.4379562,train loss=2.8445852,test loss=3.0862951
epoch=20200,train accuracy=0.6717496,test accuracy=0.43309003,train loss=2.8444211,test loss=3.0881476
epoch=20400,train accuracy=0.6717496,test accuracy=0.43552312,train loss=2.8444798,test loss=3.086594
epoch=20600,train accuracy=0.6717496,test accuracy=0.4379562,train loss=2.8443372,test loss=3.0862582
epoch=20800,train accuracy=0.6717496,test accuracy=0.4403893,train loss=2.8443494,test loss=3.0865645
epoch=21000,train accuracy=0.6717496,test accuracy=0.4379562,train loss=2.8443313,test loss=3.0864682
epoch=21200,train accuracy=0.6717496,test accuracy=0.4379562,train loss=2.844289,test loss=3.0873737
epoch=21400,train accuracy=0.6717496,test accuracy=0.4379562,train loss=2.8441837,test loss=3.0860925
epoch=21600,train accuracy=0.6717496,test accuracy=0.4379562,train loss=2.8441641,test loss=3.085744
epoch=21800,train accuracy=0.6717496,test accuracy=0.4379562,train loss=2.8440633,test loss=3.0846636
epoch=22000,train accuracy=0.6717496,test accuracy=0.43552312,train loss=2.8436856,test loss=3.0845637
epoch=22200,train accuracy=0.67255217,test accuracy=0.4379562,train loss=2.8433285,test loss=3.0870883
epoch=22400,train accuracy=0.67255217,test accuracy=0.4452555,train loss=2.8431387,test loss=3.0824013
epoch=22600,train accuracy=0.67335474,test accuracy=0.4428224,train loss=2.8424392,test loss=3.0822847
epoch=22800,train accuracy=0.67335474,test accuracy=0.4428224,train loss=2.842369,test loss=3.084192
epoch=23000,train accuracy=0.67335474,test accuracy=0.4379562,train loss=2.8423505,test loss=3.0859556
epoch=23200,train accuracy=0.67335474,test accuracy=0.4428224,train loss=2.8423467,test loss=3.0833864
epoch=23400,train accuracy=0.67335474,test accuracy=0.4379562,train loss=2.842227,test loss=3.0839274
epoch=23600,train accuracy=0.6741573,test accuracy=0.4379562,train loss=2.841583,test loss=3.0834258
epoch=23800,train accuracy=0.6741573,test accuracy=0.44768855,train loss=2.8414311,test loss=3.0797372
epoch=24000,train accuracy=0.6749599,test accuracy=0.4452555,train loss=2.8411767,test loss=3.0800555
epoch=24200,train accuracy=0.6749599,test accuracy=0.4428224,train loss=2.8409145,test loss=3.0812213
epoch=24400,train accuracy=0.6749599,test accuracy=0.4452555,train loss=2.8406396,test loss=3.0788853
epoch=24600,train accuracy=0.6757624,test accuracy=0.4428224,train loss=2.8403828,test loss=3.0799563
epoch=24800,train accuracy=0.6757624,test accuracy=0.4452555,train loss=2.8401363,test loss=3.080536
epoch=25000,train accuracy=0.6757624,test accuracy=0.4428224,train loss=2.839903,test loss=3.0809152
epoch=25200,train accuracy=0.6757624,test accuracy=0.4428224,train loss=2.8397443,test loss=3.0800197
epoch=25400,train accuracy=0.6757624,test accuracy=0.4428224,train loss=2.8394313,test loss=3.0809762
epoch=25600,train accuracy=0.676565,test accuracy=0.4403893,train loss=2.8391635,test loss=3.081243
epoch=25800,train accuracy=0.67736757,test accuracy=0.4428224,train loss=2.838738,test loss=3.0812302
epoch=26000,train accuracy=0.67736757,test accuracy=0.4403893,train loss=2.8385499,test loss=3.0795648
epoch=26200,train accuracy=0.67736757,test accuracy=0.4403893,train loss=2.8384194,test loss=3.082325
epoch=26400,train accuracy=0.67736757,test accuracy=0.4428224,train loss=2.8383796,test loss=3.0807939
epoch=26600,train accuracy=0.67736757,test accuracy=0.4403893,train loss=2.8382738,test loss=3.0798943
epoch=26800,train accuracy=0.67736757,test accuracy=0.4428224,train loss=2.8382666,test loss=3.080296
epoch=27000,train accuracy=0.67736757,test accuracy=0.4403893,train loss=2.8382325,test loss=3.0812976
epoch=27200,train accuracy=0.67736757,test accuracy=0.4379562,train loss=2.8380954,test loss=3.0829256
epoch=27400,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.837824,test loss=3.082374
epoch=27600,train accuracy=0.67817014,test accuracy=0.4403893,train loss=2.8375793,test loss=3.081406
epoch=27800,train accuracy=0.67817014,test accuracy=0.43065694,train loss=2.8374612,test loss=3.0850315
epoch=28000,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8374329,test loss=3.0834756
epoch=28200,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.837415,test loss=3.0837545
epoch=28400,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8374414,test loss=3.0827827
epoch=28600,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.837363,test loss=3.0834875
epoch=28800,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.83737,test loss=3.0823205
epoch=29000,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8373425,test loss=3.0838637
epoch=29200,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8373387,test loss=3.083257
epoch=29400,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.8372872,test loss=3.0834334
epoch=29600,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8373551,test loss=3.0833628
epoch=29800,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.837298,test loss=3.0838482
epoch=30000,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.8372915,test loss=3.08218
epoch=30200,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8373067,test loss=3.0835593
epoch=30400,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8372877,test loss=3.08254
epoch=30600,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.8372836,test loss=3.0817916
epoch=30800,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.8372707,test loss=3.081994
epoch=31000,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8372622,test loss=3.0830703
epoch=31200,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.8372557,test loss=3.081855
epoch=31400,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.8372316,test loss=3.0825424
epoch=31600,train accuracy=0.67817014,test accuracy=0.43065694,train loss=2.837244,test loss=3.0837605
epoch=31800,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.8372743,test loss=3.0829828
epoch=32000,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.8372507,test loss=3.082731
epoch=32200,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.8372445,test loss=3.08247
epoch=32400,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8371837,test loss=3.0817015
epoch=32600,train accuracy=0.67817014,test accuracy=0.43065694,train loss=2.8371575,test loss=3.083717
epoch=32800,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.8372178,test loss=3.0824804
epoch=33000,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.83715,test loss=3.083531
epoch=33200,train accuracy=0.67817014,test accuracy=0.42822385,train loss=2.8371675,test loss=3.0829473
epoch=33400,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8371522,test loss=3.0828915
epoch=33600,train accuracy=0.67817014,test accuracy=0.43065694,train loss=2.8371217,test loss=3.0840921
epoch=33800,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.83711,test loss=3.083032
epoch=34000,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.8371162,test loss=3.0819168
epoch=34200,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.8371346,test loss=3.0830865
epoch=34400,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.8371303,test loss=3.0829089
epoch=34600,train accuracy=0.67817014,test accuracy=0.42822385,train loss=2.837129,test loss=3.082721
epoch=34800,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8371289,test loss=3.0809536
epoch=35000,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8370848,test loss=3.0832002
epoch=35200,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8370774,test loss=3.082903
epoch=35400,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.837056,test loss=3.0828686
epoch=35600,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.837085,test loss=3.0827303
epoch=35800,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.8370724,test loss=3.0834537
epoch=36000,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.8370717,test loss=3.081226
epoch=36200,train accuracy=0.67817014,test accuracy=0.4379562,train loss=2.837077,test loss=3.0812724
epoch=36400,train accuracy=0.67817014,test accuracy=0.43309003,train loss=2.8371084,test loss=3.0821335
epoch=36600,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8370965,test loss=3.0815163
epoch=36800,train accuracy=0.67817014,test accuracy=0.42822385,train loss=2.837067,test loss=3.083199
epoch=37000,train accuracy=0.67817014,test accuracy=0.43065694,train loss=2.8370762,test loss=3.0817726
epoch=37200,train accuracy=0.67817014,test accuracy=0.43065694,train loss=2.837043,test loss=3.0823264
epoch=37400,train accuracy=0.67817014,test accuracy=0.42822385,train loss=2.8370862,test loss=3.0825868
epoch=37600,train accuracy=0.67817014,test accuracy=0.43065694,train loss=2.837081,test loss=3.0823734
epoch=37800,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8370876,test loss=3.0806482
epoch=38000,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.837003,test loss=3.0804949
epoch=38200,train accuracy=0.67817014,test accuracy=0.43065694,train loss=2.8371067,test loss=3.0799012
epoch=38400,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8369951,test loss=3.079012
epoch=38600,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.8369582,test loss=3.080536
epoch=38800,train accuracy=0.67817014,test accuracy=0.43552312,train loss=2.836754,test loss=3.080268
epoch=39000,train accuracy=0.6789727,test accuracy=0.43552312,train loss=2.8364742,test loss=3.0789254
epoch=39200,train accuracy=0.6789727,test accuracy=0.43552312,train loss=2.8363528,test loss=3.0789688
epoch=39400,train accuracy=0.6789727,test accuracy=0.4403893,train loss=2.8361409,test loss=3.0774708
epoch=39600,train accuracy=0.6789727,test accuracy=0.43552312,train loss=2.8360856,test loss=3.0779815
epoch=39800,train accuracy=0.6789727,test accuracy=0.4403893,train loss=2.835928,test loss=3.077453
epoch=40000,train accuracy=0.6797753,test accuracy=0.4379562,train loss=2.835713,test loss=3.0757258
epoch=40200,train accuracy=0.6797753,test accuracy=0.4428224,train loss=2.8353372,test loss=3.0739253
epoch=40400,train accuracy=0.6805779,test accuracy=0.4403893,train loss=2.8341742,test loss=3.0747628
epoch=40600,train accuracy=0.68218297,test accuracy=0.4403893,train loss=2.833653,test loss=3.0753765
epoch=40800,train accuracy=0.68218297,test accuracy=0.4428224,train loss=2.8332472,test loss=3.0736873
epoch=41000,train accuracy=0.6813804,test accuracy=0.4403893,train loss=2.8334894,test loss=3.0740616
epoch=41200,train accuracy=0.68298554,test accuracy=0.4428224,train loss=2.8329914,test loss=3.073948
epoch=41400,train accuracy=0.68298554,test accuracy=0.4379562,train loss=2.8325567,test loss=3.075111
epoch=41600,train accuracy=0.68298554,test accuracy=0.4403893,train loss=2.8325412,test loss=3.0739787
epoch=41800,train accuracy=0.68298554,test accuracy=0.4403893,train loss=2.8323927,test loss=3.0740807
epoch=42000,train accuracy=0.68298554,test accuracy=0.4428224,train loss=2.8323503,test loss=3.0737047
epoch=42200,train accuracy=0.68298554,test accuracy=0.4403893,train loss=2.8323944,test loss=3.074239
epoch=42400,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.832431,test loss=3.0742154
epoch=42600,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8323264,test loss=3.0732338
epoch=42800,train accuracy=0.68298554,test accuracy=0.4403893,train loss=2.8323975,test loss=3.0749683
epoch=43000,train accuracy=0.68298554,test accuracy=0.44768855,train loss=2.832265,test loss=3.0732834
epoch=43200,train accuracy=0.68298554,test accuracy=0.4428224,train loss=2.8322651,test loss=3.072832
epoch=43400,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8322206,test loss=3.0730717
epoch=43600,train accuracy=0.68298554,test accuracy=0.4428224,train loss=2.8322601,test loss=3.0731342
epoch=43800,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8322194,test loss=3.0732036
epoch=44000,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8322384,test loss=3.073791
epoch=44200,train accuracy=0.68298554,test accuracy=0.44768855,train loss=2.8322012,test loss=3.0733316
epoch=44400,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8322053,test loss=3.0741954
epoch=44600,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8322136,test loss=3.0731127
epoch=44800,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8322167,test loss=3.0741768
epoch=45000,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8322225,test loss=3.0731685
epoch=45200,train accuracy=0.68298554,test accuracy=0.44768855,train loss=2.8321853,test loss=3.0727506
epoch=45400,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8322022,test loss=3.0739639
epoch=45600,train accuracy=0.68298554,test accuracy=0.4452555,train loss=2.8321743,test loss=3.0732589
epoch=45800,train accuracy=0.68298554,test accuracy=0.44768855,train loss=2.8321395,test loss=3.0735161
epoch=46000,train accuracy=0.68298554,test accuracy=0.45012164,train loss=2.8321002,test loss=3.0730991
epoch=46200,train accuracy=0.68298554,test accuracy=0.45255473,train loss=2.8321428,test loss=3.073705
epoch=46400,train accuracy=0.68298554,test accuracy=0.45255473,train loss=2.8320715,test loss=3.0722632
epoch=46600,train accuracy=0.68298554,test accuracy=0.45255473,train loss=2.8319309,test loss=3.0737596
epoch=46800,train accuracy=0.68298554,test accuracy=0.45012164,train loss=2.8317978,test loss=3.0727923
epoch=47000,train accuracy=0.6837881,test accuracy=0.45012164,train loss=2.8315892,test loss=3.0731115
epoch=47200,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8315206,test loss=3.0735347
epoch=47400,train accuracy=0.6837881,test accuracy=0.45012164,train loss=2.8313951,test loss=3.0730803
epoch=47600,train accuracy=0.6837881,test accuracy=0.4452555,train loss=2.8313434,test loss=3.0736067
epoch=47800,train accuracy=0.6837881,test accuracy=0.45012164,train loss=2.8313825,test loss=3.0729496
epoch=48000,train accuracy=0.6837881,test accuracy=0.45012164,train loss=2.831314,test loss=3.0739915
epoch=48200,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8313446,test loss=3.073775
epoch=48400,train accuracy=0.6837881,test accuracy=0.45012164,train loss=2.8313422,test loss=3.0739636
epoch=48600,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8313088,test loss=3.0740137
epoch=48800,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8313076,test loss=3.0746474
epoch=49000,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312984,test loss=3.0735986
epoch=49200,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.831313,test loss=3.0738068
epoch=49400,train accuracy=0.6837881,test accuracy=0.45012164,train loss=2.8313365,test loss=3.0731716
epoch=49600,train accuracy=0.6837881,test accuracy=0.4452555,train loss=2.8313568,test loss=3.0739017
epoch=49800,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8313024,test loss=3.074033
epoch=50000,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8313785,test loss=3.0742817
epoch=50200,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.831309,test loss=3.075298
epoch=50400,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.831308,test loss=3.0745816
epoch=50600,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8313153,test loss=3.074216
epoch=50800,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8313842,test loss=3.0752168
epoch=51000,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8313115,test loss=3.0740354
epoch=51200,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.831279,test loss=3.0746188
epoch=51400,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312945,test loss=3.0745335
epoch=51600,train accuracy=0.6837881,test accuracy=0.4452555,train loss=2.8312688,test loss=3.0760908
epoch=51800,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8313258,test loss=3.0737126
epoch=52000,train accuracy=0.6837881,test accuracy=0.4452555,train loss=2.8313367,test loss=3.0749762
epoch=52200,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.831358,test loss=3.074062
epoch=52400,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312883,test loss=3.0751784
epoch=52600,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312685,test loss=3.0737548
epoch=52800,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312814,test loss=3.073683
epoch=53000,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312576,test loss=3.0745616
epoch=53200,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312366,test loss=3.074898
epoch=53400,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312697,test loss=3.0734186
epoch=53600,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312478,test loss=3.0749521
epoch=53800,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312383,test loss=3.0742252
epoch=54000,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8312166,test loss=3.0742524
epoch=54200,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.8311796,test loss=3.075201
epoch=54400,train accuracy=0.6837881,test accuracy=0.44768855,train loss=2.83098,test loss=3.0743496
epoch=54600,train accuracy=0.6845907,test accuracy=0.4452555,train loss=2.830734,test loss=3.07524
epoch=54800,train accuracy=0.6845907,test accuracy=0.4452555,train loss=2.8305788,test loss=3.0753076
epoch=55000,train accuracy=0.6845907,test accuracy=0.44768855,train loss=2.8305118,test loss=3.0743122
epoch=55200,train accuracy=0.6845907,test accuracy=0.4428224,train loss=2.8305194,test loss=3.07448
epoch=55400,train accuracy=0.6845907,test accuracy=0.4452555,train loss=2.8304918,test loss=3.07405
epoch=55600,train accuracy=0.6845907,test accuracy=0.44768855,train loss=2.8304708,test loss=3.0730183
epoch=55800,train accuracy=0.6845907,test accuracy=0.4428224,train loss=2.830469,test loss=3.075416
epoch=56000,train accuracy=0.6845907,test accuracy=0.4428224,train loss=2.830469,test loss=3.0737367
epoch=56200,train accuracy=0.6845907,test accuracy=0.4452555,train loss=2.8304946,test loss=3.075387
epoch=56400,train accuracy=0.6845907,test accuracy=0.4452555,train loss=2.8304467,test loss=3.0744965
epoch=56600,train accuracy=0.6845907,test accuracy=0.4428224,train loss=2.8305066,test loss=3.0746312
epoch=56800,train accuracy=0.6845907,test accuracy=0.4428224,train loss=2.830427,test loss=3.0745392
epoch=57000,train accuracy=0.6845907,test accuracy=0.4428224,train loss=2.8304734,test loss=3.0749962
epoch=57200,train accuracy=0.6845907,test accuracy=0.4428224,train loss=2.8304195,test loss=3.0752218
epoch=57400,train accuracy=0.6845907,test accuracy=0.4428224,train loss=2.8302956,test loss=3.0751143
epoch=57600,train accuracy=0.6845907,test accuracy=0.4428224,train loss=2.830298,test loss=3.07495
epoch=57800,train accuracy=0.6853933,test accuracy=0.4428224,train loss=2.8300357,test loss=3.0758579
epoch=58000,train accuracy=0.6853933,test accuracy=0.4379562,train loss=2.8299065,test loss=3.0759878
epoch=58200,train accuracy=0.6853933,test accuracy=0.4403893,train loss=2.8296664,test loss=3.0760098
epoch=58400,train accuracy=0.68619585,test accuracy=0.4428224,train loss=2.8296552,test loss=3.0760798
epoch=58600,train accuracy=0.68619585,test accuracy=0.4403893,train loss=2.8294046,test loss=3.0760267
epoch=58800,train accuracy=0.68619585,test accuracy=0.4403893,train loss=2.8291771,test loss=3.0773542
epoch=59000,train accuracy=0.68619585,test accuracy=0.4379562,train loss=2.8292308,test loss=3.0765574
epoch=59200,train accuracy=0.68619585,test accuracy=0.4403893,train loss=2.829098,test loss=3.0762699
epoch=59400,train accuracy=0.68619585,test accuracy=0.43552312,train loss=2.8288605,test loss=3.0784621
epoch=59600,train accuracy=0.68699837,test accuracy=0.43309003,train loss=2.8285263,test loss=3.0795758
epoch=59800,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.828223,test loss=3.081258
epoch=60000,train accuracy=0.68699837,test accuracy=0.43309003,train loss=2.8280892,test loss=3.081277
epoch=60200,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.8281431,test loss=3.0814836
epoch=60400,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.8280728,test loss=3.0808272
epoch=60600,train accuracy=0.68699837,test accuracy=0.43065694,train loss=2.8280568,test loss=3.0814693
epoch=60800,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.8280776,test loss=3.0803578
epoch=61000,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.8280425,test loss=3.0816154
epoch=61200,train accuracy=0.68699837,test accuracy=0.43309003,train loss=2.8280442,test loss=3.0806394
epoch=61400,train accuracy=0.68699837,test accuracy=0.43309003,train loss=2.8280454,test loss=3.0815911
epoch=61600,train accuracy=0.68699837,test accuracy=0.43309003,train loss=2.8280134,test loss=3.0802748
epoch=61800,train accuracy=0.68699837,test accuracy=0.43309003,train loss=2.8280542,test loss=3.0812638
epoch=62000,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.8280094,test loss=3.0804756
epoch=62200,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.8280306,test loss=3.0807722
epoch=62400,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.828009,test loss=3.080165
epoch=62600,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.8280067,test loss=3.0796685
epoch=62800,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.8279667,test loss=3.0798416
epoch=63000,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.827909,test loss=3.0793722
epoch=63200,train accuracy=0.68699837,test accuracy=0.43552312,train loss=2.8277955,test loss=3.079147
epoch=63400,train accuracy=0.68699837,test accuracy=0.43309003,train loss=2.8276405,test loss=3.0821505
epoch=63600,train accuracy=0.68780094,test accuracy=0.4379562,train loss=2.8275042,test loss=3.0803063
epoch=63800,train accuracy=0.68780094,test accuracy=0.4403893,train loss=2.8273628,test loss=3.0794957
epoch=64000,train accuracy=0.68780094,test accuracy=0.4379562,train loss=2.827265,test loss=3.080073
epoch=64200,train accuracy=0.68780094,test accuracy=0.43309003,train loss=2.8271637,test loss=3.0802996
epoch=64400,train accuracy=0.68780094,test accuracy=0.43065694,train loss=2.827202,test loss=3.0812345
epoch=64600,train accuracy=0.68780094,test accuracy=0.43309003,train loss=2.8271437,test loss=3.0802538
epoch=64800,train accuracy=0.68780094,test accuracy=0.43309003,train loss=2.8271334,test loss=3.079667
epoch=65000,train accuracy=0.68780094,test accuracy=0.43309003,train loss=2.827035,test loss=3.0806873
epoch=65200,train accuracy=0.68780094,test accuracy=0.43552312,train loss=2.8268895,test loss=3.080366
epoch=65400,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.8267844,test loss=3.0803175
epoch=65600,train accuracy=0.6886035,test accuracy=0.43309003,train loss=2.8266287,test loss=3.081095
epoch=65800,train accuracy=0.6886035,test accuracy=0.43065694,train loss=2.826496,test loss=3.0807028
epoch=66000,train accuracy=0.6886035,test accuracy=0.43065694,train loss=2.826427,test loss=3.08259
epoch=66200,train accuracy=0.6886035,test accuracy=0.43309003,train loss=2.8263996,test loss=3.0818284
epoch=66400,train accuracy=0.6886035,test accuracy=0.43309003,train loss=2.8264384,test loss=3.0812473
epoch=66600,train accuracy=0.6886035,test accuracy=0.43552312,train loss=2.8263848,test loss=3.0821483
epoch=66800,train accuracy=0.6886035,test accuracy=0.43552312,train loss=2.826333,test loss=3.0820336
epoch=67000,train accuracy=0.6886035,test accuracy=0.43309003,train loss=2.8264043,test loss=3.0824735
epoch=67200,train accuracy=0.6886035,test accuracy=0.43309003,train loss=2.8264077,test loss=3.0815282
epoch=67400,train accuracy=0.6886035,test accuracy=0.4403893,train loss=2.8263702,test loss=3.0801804
epoch=67600,train accuracy=0.6886035,test accuracy=0.43552312,train loss=2.8264117,test loss=3.08047
epoch=67800,train accuracy=0.6886035,test accuracy=0.43552312,train loss=2.8263776,test loss=3.080077
epoch=68000,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.8263764,test loss=3.080863
epoch=68200,train accuracy=0.6886035,test accuracy=0.43552312,train loss=2.8263826,test loss=3.0816262
epoch=68400,train accuracy=0.6886035,test accuracy=0.43552312,train loss=2.8263881,test loss=3.0803666
epoch=68600,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.8263738,test loss=3.081143
epoch=68800,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.8263707,test loss=3.0802748
epoch=69000,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.826372,test loss=3.0824504
epoch=69200,train accuracy=0.6886035,test accuracy=0.43552312,train loss=2.8263626,test loss=3.0806024
epoch=69400,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.8263571,test loss=3.0791194
epoch=69600,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.8263521,test loss=3.0809298
epoch=69800,train accuracy=0.6886035,test accuracy=0.4403893,train loss=2.8263423,test loss=3.079707
epoch=70000,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.8263535,test loss=3.0796258
epoch=70200,train accuracy=0.6886035,test accuracy=0.4428224,train loss=2.826305,test loss=3.0792673
epoch=70400,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.8262877,test loss=3.0803728
epoch=70600,train accuracy=0.6886035,test accuracy=0.4379562,train loss=2.826259,test loss=3.0800235
epoch=70800,train accuracy=0.6886035,test accuracy=0.4428224,train loss=2.8262475,test loss=3.0791943
epoch=71000,train accuracy=0.6886035,test accuracy=0.4428224,train loss=2.8262029,test loss=3.0780408
epoch=71200,train accuracy=0.6886035,test accuracy=0.4452555,train loss=2.8261275,test loss=3.0776017
epoch=71400,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.8259764,test loss=3.0787244
epoch=71600,train accuracy=0.6894061,test accuracy=0.4403893,train loss=2.8259368,test loss=3.079231
epoch=71800,train accuracy=0.6894061,test accuracy=0.4403893,train loss=2.8258164,test loss=3.0787005
epoch=72000,train accuracy=0.6894061,test accuracy=0.44768855,train loss=2.825771,test loss=3.0765505
epoch=72200,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.825695,test loss=3.0758052
epoch=72400,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.825666,test loss=3.076862
epoch=72600,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.82563,test loss=3.0770779
epoch=72800,train accuracy=0.6894061,test accuracy=0.4403893,train loss=2.825615,test loss=3.0778484
epoch=73000,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.8256128,test loss=3.0769625
epoch=73200,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.8255782,test loss=3.0763364
epoch=73400,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.825613,test loss=3.0774105
epoch=73600,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.8255897,test loss=3.0766168
epoch=73800,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.8255813,test loss=3.0777128
epoch=74000,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.82556,test loss=3.0777059
epoch=74200,train accuracy=0.6894061,test accuracy=0.4403893,train loss=2.825555,test loss=3.0779338
epoch=74400,train accuracy=0.6894061,test accuracy=0.4403893,train loss=2.8255854,test loss=3.0778482
epoch=74600,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.8255632,test loss=3.0774872
epoch=74800,train accuracy=0.6894061,test accuracy=0.4403893,train loss=2.8255742,test loss=3.077955
epoch=75000,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.8255746,test loss=3.0781984
epoch=75200,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.8255982,test loss=3.07728
epoch=75400,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.8255699,test loss=3.0775409
epoch=75600,train accuracy=0.6894061,test accuracy=0.44768855,train loss=2.8255675,test loss=3.0774188
epoch=75800,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.8256092,test loss=3.0781674
epoch=76000,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.8255327,test loss=3.0775633
epoch=76200,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.8255906,test loss=3.0766397
epoch=76400,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.825612,test loss=3.0780268
epoch=76600,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.8256567,test loss=3.0776346
epoch=76800,train accuracy=0.6894061,test accuracy=0.4403893,train loss=2.825535,test loss=3.0775797
epoch=77000,train accuracy=0.6894061,test accuracy=0.4452555,train loss=2.8255453,test loss=3.0766008
epoch=77200,train accuracy=0.6894061,test accuracy=0.4428224,train loss=2.8255572,test loss=3.0765948
epoch=77400,train accuracy=0.6894061,test accuracy=0.4403893,train loss=2.825556,test loss=3.0775583
epoch=77600,train accuracy=0.6894061,test accuracy=0.4403893,train loss=2.8255093,test loss=3.0773492
epoch=77800,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.825065,test loss=3.0780883
epoch=78000,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.8247988,test loss=3.0775695
epoch=78200,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.824745,test loss=3.0757608
epoch=78400,train accuracy=0.6902087,test accuracy=0.4452555,train loss=2.8247619,test loss=3.0760813
epoch=78600,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8247309,test loss=3.0763946
epoch=78800,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.8247266,test loss=3.0764668
epoch=79000,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8247538,test loss=3.0762978
epoch=79200,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.8247786,test loss=3.0760846
epoch=79400,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.824763,test loss=3.0762398
epoch=79600,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.8247437,test loss=3.0770867
epoch=79800,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.8247666,test loss=3.0765576
epoch=80000,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8247185,test loss=3.076565
epoch=80200,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8247306,test loss=3.0765421
epoch=80400,train accuracy=0.6902087,test accuracy=0.4452555,train loss=2.824739,test loss=3.0754998
epoch=80600,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8247273,test loss=3.0770855
epoch=80800,train accuracy=0.6902087,test accuracy=0.4452555,train loss=2.8247595,test loss=3.0755057
epoch=81000,train accuracy=0.6902087,test accuracy=0.4452555,train loss=2.8247046,test loss=3.0752707
epoch=81200,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8247275,test loss=3.0762904
epoch=81400,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.824756,test loss=3.0762808
epoch=81600,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.824726,test loss=3.075807
epoch=81800,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8247123,test loss=3.076397
epoch=82000,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.824713,test loss=3.0755312
epoch=82200,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.824733,test loss=3.0767891
epoch=82400,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.824745,test loss=3.0765889
epoch=82600,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.8247123,test loss=3.0765934
epoch=82800,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8247135,test loss=3.0750124
epoch=83000,train accuracy=0.6902087,test accuracy=0.4452555,train loss=2.8247228,test loss=3.0760047
epoch=83200,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.824755,test loss=3.0759087
epoch=83400,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.8247042,test loss=3.0762227
epoch=83600,train accuracy=0.6902087,test accuracy=0.4452555,train loss=2.8247025,test loss=3.0754516
epoch=83800,train accuracy=0.6902087,test accuracy=0.4452555,train loss=2.8246903,test loss=3.0756948
epoch=84000,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.82469,test loss=3.0754192
epoch=84200,train accuracy=0.6902087,test accuracy=0.4452555,train loss=2.8246903,test loss=3.0750601
epoch=84400,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8246934,test loss=3.0758326
epoch=84600,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.824687,test loss=3.0755801
epoch=84800,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8246849,test loss=3.0756423
epoch=85000,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8247013,test loss=3.0758939
epoch=85200,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.8247006,test loss=3.0763571
epoch=85400,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.8246868,test loss=3.0758827
epoch=85600,train accuracy=0.6902087,test accuracy=0.4403893,train loss=2.8245735,test loss=3.07585
epoch=85800,train accuracy=0.6902087,test accuracy=0.4428224,train loss=2.824408,test loss=3.0753093
epoch=86000,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8241837,test loss=3.0748317
epoch=86200,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8240068,test loss=3.0745108
epoch=86400,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8239448,test loss=3.0739775
epoch=86600,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8239043,test loss=3.0745895
epoch=86800,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8239093,test loss=3.074694
epoch=87000,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8238955,test loss=3.0747676
epoch=87200,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.823889,test loss=3.0748305
epoch=87400,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8238988,test loss=3.07454
epoch=87600,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8238769,test loss=3.0744991
epoch=87800,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8238988,test loss=3.0744705
epoch=88000,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8238995,test loss=3.0750613
epoch=88200,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8239026,test loss=3.0745535
epoch=88400,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8238788,test loss=3.0744312
epoch=88600,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8238943,test loss=3.0752132
epoch=88800,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8238792,test loss=3.074172
epoch=89000,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8238993,test loss=3.0766537
epoch=89200,train accuracy=0.69101125,test accuracy=0.44768855,train loss=2.8238783,test loss=3.0734742
epoch=89400,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.823907,test loss=3.0755312
epoch=89600,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8238895,test loss=3.0748584
epoch=89800,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8238993,test loss=3.074132
epoch=90000,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.823897,test loss=3.0747242
epoch=90200,train accuracy=0.69101125,test accuracy=0.4403893,train loss=2.8239036,test loss=3.07507
epoch=90400,train accuracy=0.69101125,test accuracy=0.4379562,train loss=2.8238914,test loss=3.0759497
epoch=90600,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8239045,test loss=3.0757227
epoch=90800,train accuracy=0.69101125,test accuracy=0.4403893,train loss=2.823886,test loss=3.0759592
epoch=91000,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8238785,test loss=3.0746398
epoch=91200,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8238697,test loss=3.0748742
epoch=91400,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8238897,test loss=3.0739756
epoch=91600,train accuracy=0.69101125,test accuracy=0.4403893,train loss=2.8239102,test loss=3.0749404
epoch=91800,train accuracy=0.69101125,test accuracy=0.4428224,train loss=2.8238647,test loss=3.0758893
epoch=92000,train accuracy=0.69101125,test accuracy=0.4403893,train loss=2.8238454,test loss=3.0760753
epoch=92200,train accuracy=0.69101125,test accuracy=0.4452555,train loss=2.8238463,test loss=3.0738986
epoch=92400,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8235495,test loss=3.074758
epoch=92600,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.82319,test loss=3.0746458
epoch=92800,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8231215,test loss=3.0740423
epoch=93000,train accuracy=0.6918138,test accuracy=0.44768855,train loss=2.8231294,test loss=3.0733795
epoch=93200,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230915,test loss=3.0739133
epoch=93400,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.823102,test loss=3.0735588
epoch=93600,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8231213,test loss=3.0744483
epoch=93800,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.823074,test loss=3.0732083
epoch=94000,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230824,test loss=3.0730426
epoch=94200,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8231063,test loss=3.0731037
epoch=94400,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230686,test loss=3.0747037
epoch=94600,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.823082,test loss=3.0746114
epoch=94800,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230798,test loss=3.0734959
epoch=95000,train accuracy=0.6918138,test accuracy=0.4403893,train loss=2.823098,test loss=3.075574
epoch=95200,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230946,test loss=3.0736425
epoch=95400,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230646,test loss=3.073747
epoch=95600,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230767,test loss=3.073201
epoch=95800,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230739,test loss=3.0738862
epoch=96000,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230686,test loss=3.0735865
epoch=96200,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230484,test loss=3.072579
epoch=96400,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230877,test loss=3.0738297
epoch=96600,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.823084,test loss=3.0740001
epoch=96800,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.823081,test loss=3.073639
epoch=97000,train accuracy=0.6918138,test accuracy=0.44768855,train loss=2.823039,test loss=3.0721738
epoch=97200,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230538,test loss=3.0734982
epoch=97400,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230667,test loss=3.0728736
epoch=97600,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230586,test loss=3.0745258
epoch=97800,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.823058,test loss=3.07257
epoch=98000,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230476,test loss=3.0716932
epoch=98200,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230515,test loss=3.0729356
epoch=98400,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230703,test loss=3.0742643
epoch=98600,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.823086,test loss=3.073249
epoch=98800,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230674,test loss=3.0733945
epoch=99000,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8231041,test loss=3.0726619
epoch=99200,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230665,test loss=3.0745306
epoch=99400,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.823057,test loss=3.07367
epoch=99600,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.8230605,test loss=3.0726414
epoch=99800,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230495,test loss=3.0738366
epoch=100000,train accuracy=0.6918138,test accuracy=0.4428224,train loss=2.8230817,test loss=3.0738645
epoch=100200,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.823045,test loss=3.0738962
epoch=100400,train accuracy=0.6918138,test accuracy=0.4452555,train loss=2.823046,test loss=3.0737453
epoch=100600,train accuracy=0.6918138,test accuracy=0.44768855,train loss=2.8228285,test loss=3.072081
epoch=100800,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8225553,test loss=3.0730495
epoch=101000,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8223426,test loss=3.0725558
epoch=101200,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8222842,test loss=3.0731142
epoch=101400,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8222756,test loss=3.0729132
epoch=101600,train accuracy=0.69261634,test accuracy=0.4428224,train loss=2.8222752,test loss=3.0722878
epoch=101800,train accuracy=0.69261634,test accuracy=0.44768855,train loss=2.822265,test loss=3.070881
epoch=102000,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8222837,test loss=3.072823
epoch=102200,train accuracy=0.69261634,test accuracy=0.44768855,train loss=2.8223202,test loss=3.070954
epoch=102400,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8223104,test loss=3.0709713
epoch=102600,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8223078,test loss=3.0716815
epoch=102800,train accuracy=0.69261634,test accuracy=0.44768855,train loss=2.8223414,test loss=3.0702777
epoch=103000,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8222442,test loss=3.071608
epoch=103200,train accuracy=0.69261634,test accuracy=0.44768855,train loss=2.822275,test loss=3.0707388
epoch=103400,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8222733,test loss=3.0704758
epoch=103600,train accuracy=0.69261634,test accuracy=0.44768855,train loss=2.822272,test loss=3.070375
epoch=103800,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8222454,test loss=3.070346
epoch=104000,train accuracy=0.69261634,test accuracy=0.44768855,train loss=2.822241,test loss=3.0701575
epoch=104200,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8222823,test loss=3.0711517
epoch=104400,train accuracy=0.69261634,test accuracy=0.44768855,train loss=2.8222415,test loss=3.0713544
epoch=104600,train accuracy=0.69261634,test accuracy=0.44768855,train loss=2.8222244,test loss=3.0708432
epoch=104800,train accuracy=0.69261634,test accuracy=0.4452555,train loss=2.8222616,test loss=3.071333
epoch=105000,train accuracy=0.69261634,test accuracy=0.4428224,train loss=2.8222191,test loss=3.0720108
epoch=105200,train accuracy=0.69261634,test accuracy=0.4428224,train loss=2.8221645,test loss=3.0718305
epoch=105400,train accuracy=0.69261634,test accuracy=0.4428224,train loss=2.822008,test loss=3.0714912
epoch=105600,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8218374,test loss=3.0726504
epoch=105800,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8216684,test loss=3.0724254
epoch=106000,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.821559,test loss=3.0713556
epoch=106200,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.821534,test loss=3.071058
epoch=106400,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.821467,test loss=3.0730145
epoch=106600,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214724,test loss=3.0713608
epoch=106800,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214707,test loss=3.071462
epoch=107000,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214576,test loss=3.0710018
epoch=107200,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214688,test loss=3.0731688
epoch=107400,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214672,test loss=3.0729969
epoch=107600,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214524,test loss=3.07211
epoch=107800,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.821453,test loss=3.073663
epoch=108000,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.8214607,test loss=3.0723245
epoch=108200,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.821462,test loss=3.0736048
epoch=108400,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8214722,test loss=3.0737212
epoch=108600,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8214767,test loss=3.073876
epoch=108800,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.821493,test loss=3.0725794
epoch=109000,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.8214595,test loss=3.0731914
epoch=109200,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.8214304,test loss=3.0721402
epoch=109400,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214579,test loss=3.0739832
epoch=109600,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.821482,test loss=3.0735955
epoch=109800,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.82146,test loss=3.0751498
epoch=110000,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.8214865,test loss=3.0724843
epoch=110200,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214445,test loss=3.0741293
epoch=110400,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214812,test loss=3.074944
epoch=110600,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8215225,test loss=3.0739286
epoch=110800,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.8214645,test loss=3.0722363
epoch=111000,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.821491,test loss=3.073931
epoch=111200,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.8214855,test loss=3.0741653
epoch=111400,train accuracy=0.6934189,test accuracy=0.43309003,train loss=2.821455,test loss=3.075077
epoch=111600,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214788,test loss=3.0753891
epoch=111800,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214607,test loss=3.0752363
epoch=112000,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.821459,test loss=3.0752099
epoch=112200,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214538,test loss=3.073764
epoch=112400,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.8214767,test loss=3.0732088
epoch=112600,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214848,test loss=3.0745716
epoch=112800,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214417,test loss=3.0753272
epoch=113000,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214424,test loss=3.074827
epoch=113200,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214483,test loss=3.0744052
epoch=113400,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.821508,test loss=3.0729146
epoch=113600,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214338,test loss=3.0756066
epoch=113800,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.82145,test loss=3.075197
epoch=114000,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8214633,test loss=3.0745738
epoch=114200,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8214526,test loss=3.0741441
epoch=114400,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.821444,test loss=3.0746076
epoch=114600,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214386,test loss=3.0753496
epoch=114800,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8214512,test loss=3.074265
epoch=115000,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.82148,test loss=3.0748873
epoch=115200,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.821399,test loss=3.0763898
epoch=115400,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214204,test loss=3.0761948
epoch=115600,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214312,test loss=3.075212
epoch=115800,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214633,test loss=3.073431
epoch=116000,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.821428,test loss=3.0745006
epoch=116200,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.8214345,test loss=3.074181
epoch=116400,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.82145,test loss=3.0747848
epoch=116600,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.821437,test loss=3.0746672
epoch=116800,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214524,test loss=3.0753424
epoch=117000,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821419,test loss=3.0747335
epoch=117200,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214185,test loss=3.0758083
epoch=117400,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214338,test loss=3.074372
epoch=117600,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.821425,test loss=3.0750399
epoch=117800,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214471,test loss=3.0746293
epoch=118000,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214371,test loss=3.0751734
epoch=118200,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214226,test loss=3.0745008
epoch=118400,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214543,test loss=3.0754397
epoch=118600,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214734,test loss=3.0752463
epoch=118800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821432,test loss=3.0759542
epoch=119000,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214414,test loss=3.0748386
epoch=119200,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.821425,test loss=3.0765667
epoch=119400,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214283,test loss=3.0750747
epoch=119600,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214252,test loss=3.0756748
epoch=119800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821417,test loss=3.0747736
epoch=120000,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.82141,test loss=3.0770745
epoch=120200,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.8214006,test loss=3.0738523
epoch=120400,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214538,test loss=3.0751443
epoch=120600,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214464,test loss=3.0744705
epoch=120800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214183,test loss=3.0752404
epoch=121000,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.821428,test loss=3.0763178
epoch=121200,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214002,test loss=3.0759578
epoch=121400,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214135,test loss=3.0757184
epoch=121600,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821426,test loss=3.0747857
epoch=121800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214056,test loss=3.0737085
epoch=122000,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214195,test loss=3.0751266
epoch=122200,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8214307,test loss=3.0740523
epoch=122400,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.821456,test loss=3.0750997
epoch=122600,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821435,test loss=3.0752501
epoch=122800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214197,test loss=3.0754447
epoch=123000,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821396,test loss=3.0756373
epoch=123200,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8214447,test loss=3.074634
epoch=123400,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8213954,test loss=3.075555
epoch=123600,train accuracy=0.6934189,test accuracy=0.4452555,train loss=2.821422,test loss=3.0740058
epoch=123800,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8214257,test loss=3.0747437
epoch=124000,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214128,test loss=3.0758927
epoch=124200,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821394,test loss=3.0760279
epoch=124400,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8213751,test loss=3.0751824
epoch=124600,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214335,test loss=3.0767584
epoch=124800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8213978,test loss=3.0755436
epoch=125000,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8213754,test loss=3.0748837
epoch=125200,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.821423,test loss=3.0752525
epoch=125400,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821401,test loss=3.0771098
epoch=125600,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.821405,test loss=3.075291
epoch=125800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214123,test loss=3.075836
epoch=126000,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8213978,test loss=3.0748472
epoch=126200,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.821391,test loss=3.0753157
epoch=126400,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8213673,test loss=3.075232
epoch=126600,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8213844,test loss=3.0760653
epoch=126800,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8213892,test loss=3.0758243
epoch=127000,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214064,test loss=3.0760257
epoch=127200,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214064,test loss=3.0764315
epoch=127400,train accuracy=0.6934189,test accuracy=0.43552312,train loss=2.8214195,test loss=3.0750196
epoch=127600,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214653,test loss=3.0745955
epoch=127800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214004,test loss=3.0759048
epoch=128000,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8214343,test loss=3.0758252
epoch=128200,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8213928,test loss=3.0764139
epoch=128400,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821411,test loss=3.076866
epoch=128600,train accuracy=0.6934189,test accuracy=0.4428224,train loss=2.8213787,test loss=3.0755289
epoch=128800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8214025,test loss=3.0758193
epoch=129000,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8213587,test loss=3.078065
epoch=129200,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8213837,test loss=3.077026
epoch=129400,train accuracy=0.6934189,test accuracy=0.4403893,train loss=2.8213742,test loss=3.0755196
epoch=129600,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8213005,test loss=3.0770264
epoch=129800,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.8210855,test loss=3.0764124
epoch=130000,train accuracy=0.6934189,test accuracy=0.4379562,train loss=2.821058,test loss=3.07644
epoch=130200,train accuracy=0.6942215,test accuracy=0.4403893,train loss=2.8208416,test loss=3.0745907
epoch=130400,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8207815,test loss=3.075978
epoch=130600,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.820663,test loss=3.0769463
epoch=130800,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206365,test loss=3.0766928
epoch=131000,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206975,test loss=3.0759823
epoch=131200,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206654,test loss=3.076786
epoch=131400,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206544,test loss=3.0756931
epoch=131600,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206747,test loss=3.0743952
epoch=131800,train accuracy=0.6942215,test accuracy=0.4403893,train loss=2.8206716,test loss=3.076642
epoch=132000,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206627,test loss=3.0762482
epoch=132200,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206263,test loss=3.0755546
epoch=132400,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8206367,test loss=3.077084
epoch=132600,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206484,test loss=3.0767834
epoch=132800,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206034,test loss=3.0751708
epoch=133000,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8206263,test loss=3.0761187
epoch=133200,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8206217,test loss=3.0771081
epoch=133400,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8206625,test loss=3.0748687
epoch=133600,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206134,test loss=3.0754213
epoch=133800,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8206098,test loss=3.0755665
epoch=134000,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8205855,test loss=3.0750237
epoch=134200,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206334,test loss=3.074971
epoch=134400,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.820586,test loss=3.0761333
epoch=134600,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8206134,test loss=3.0761142
epoch=134800,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8205855,test loss=3.0756664
epoch=135000,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.820616,test loss=3.075296
epoch=135200,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206224,test loss=3.074822
epoch=135400,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8205612,test loss=3.075988
epoch=135600,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8206282,test loss=3.077774
epoch=135800,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8205733,test loss=3.0755782
epoch=136000,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8206332,test loss=3.074552
epoch=136200,train accuracy=0.6942215,test accuracy=0.43552312,train loss=2.8205879,test loss=3.0749376
epoch=136400,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8205729,test loss=3.0748227
epoch=136600,train accuracy=0.6942215,test accuracy=0.4452555,train loss=2.820583,test loss=3.0736551
epoch=136800,train accuracy=0.6942215,test accuracy=0.4403893,train loss=2.8205829,test loss=3.073826
epoch=137000,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8205473,test loss=3.074651
epoch=137200,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.820549,test loss=3.073515
epoch=137400,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8205764,test loss=3.074798
epoch=137600,train accuracy=0.6942215,test accuracy=0.4403893,train loss=2.82056,test loss=3.0750875
epoch=137800,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8205624,test loss=3.0754395
epoch=138000,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8205178,test loss=3.0749004
epoch=138200,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8205447,test loss=3.0751543
epoch=138400,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8204162,test loss=3.0756896
epoch=138600,train accuracy=0.6942215,test accuracy=0.4379562,train loss=2.8202305,test loss=3.074356
epoch=138800,train accuracy=0.6950241,test accuracy=0.4379562,train loss=2.8199043,test loss=3.0760174
epoch=139000,train accuracy=0.6950241,test accuracy=0.4379562,train loss=2.8198187,test loss=3.0737283
epoch=139200,train accuracy=0.6950241,test accuracy=0.4379562,train loss=2.8197496,test loss=3.0761333
epoch=139400,train accuracy=0.6950241,test accuracy=0.43552312,train loss=2.819737,test loss=3.076799
epoch=139600,train accuracy=0.6950241,test accuracy=0.44768855,train loss=2.819777,test loss=3.073789
epoch=139800,train accuracy=0.6950241,test accuracy=0.4403893,train loss=2.8197374,test loss=3.075907
epoch=140000,train accuracy=0.6950241,test accuracy=0.43552312,train loss=2.8197029,test loss=3.0759423
epoch=140200,train accuracy=0.6950241,test accuracy=0.4379562,train loss=2.81969,test loss=3.0752666
epoch=140400,train accuracy=0.6950241,test accuracy=0.4403893,train loss=2.8197026,test loss=3.0760086
epoch=140600,train accuracy=0.6950241,test accuracy=0.43552312,train loss=2.8196445,test loss=3.0767386
epoch=140800,train accuracy=0.6950241,test accuracy=0.4379562,train loss=2.819562,test loss=3.0756257
epoch=141000,train accuracy=0.6950241,test accuracy=0.43552312,train loss=2.8192728,test loss=3.079014
epoch=141200,train accuracy=0.71428573,test accuracy=0.43309003,train loss=2.8037083,test loss=3.0795138
epoch=141400,train accuracy=0.7166934,test accuracy=0.4452555,train loss=2.7992203,test loss=3.0721567
epoch=141600,train accuracy=0.7166934,test accuracy=0.44768855,train loss=2.7986467,test loss=3.0698059
epoch=141800,train accuracy=0.71829855,test accuracy=0.45012164,train loss=2.7973833,test loss=3.0702493
epoch=142000,train accuracy=0.7199037,test accuracy=0.45255473,train loss=2.7959263,test loss=3.0685856
epoch=142200,train accuracy=0.7199037,test accuracy=0.45498782,train loss=2.795187,test loss=3.0658526
epoch=142400,train accuracy=0.7215088,test accuracy=0.45742092,train loss=2.7940195,test loss=3.0652065
epoch=142600,train accuracy=0.7215088,test accuracy=0.45742092,train loss=2.7933228,test loss=3.0626292
epoch=142800,train accuracy=0.72311395,test accuracy=0.4647202,train loss=2.7924385,test loss=3.058272
epoch=143000,train accuracy=0.72311395,test accuracy=0.459854,train loss=2.7922206,test loss=3.0571985
epoch=143200,train accuracy=0.72311395,test accuracy=0.4622871,train loss=2.7921627,test loss=3.0559936
epoch=143400,train accuracy=0.72311395,test accuracy=0.4647202,train loss=2.7920706,test loss=3.0546947
epoch=143600,train accuracy=0.72311395,test accuracy=0.46958637,train loss=2.7919166,test loss=3.0532806
epoch=143800,train accuracy=0.72391653,test accuracy=0.46958637,train loss=2.791736,test loss=3.053967
epoch=144000,train accuracy=0.72391653,test accuracy=0.46958637,train loss=2.7915578,test loss=3.053721
epoch=144200,train accuracy=0.72391653,test accuracy=0.46958637,train loss=2.791429,test loss=3.0538344
epoch=144400,train accuracy=0.72391653,test accuracy=0.46958637,train loss=2.7913268,test loss=3.0529783
epoch=144600,train accuracy=0.7247191,test accuracy=0.46958637,train loss=2.7911096,test loss=3.0552757
epoch=144800,train accuracy=0.7247191,test accuracy=0.46958637,train loss=2.7907727,test loss=3.052572
epoch=145000,train accuracy=0.7247191,test accuracy=0.47201946,train loss=2.7904894,test loss=3.0531235
epoch=145200,train accuracy=0.7255217,test accuracy=0.46958637,train loss=2.7902532,test loss=3.051922
epoch=145400,train accuracy=0.7255217,test accuracy=0.46715328,train loss=2.7901032,test loss=3.0524914
epoch=145600,train accuracy=0.7255217,test accuracy=0.46715328,train loss=2.790154,test loss=3.052137
epoch=145800,train accuracy=0.7255217,test accuracy=0.46715328,train loss=2.7901018,test loss=3.0542934
epoch=146000,train accuracy=0.7255217,test accuracy=0.46958637,train loss=2.7900004,test loss=3.0526497
epoch=146200,train accuracy=0.7255217,test accuracy=0.46958637,train loss=2.7899902,test loss=3.0526428
epoch=146400,train accuracy=0.7255217,test accuracy=0.4622871,train loss=2.7899685,test loss=3.053621
epoch=146600,train accuracy=0.7255217,test accuracy=0.47201946,train loss=2.789898,test loss=3.052607
epoch=146800,train accuracy=0.7255217,test accuracy=0.47201946,train loss=2.7894866,test loss=3.0526464
epoch=147000,train accuracy=0.72632426,test accuracy=0.47201946,train loss=2.7890062,test loss=3.0524542
epoch=147200,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.788513,test loss=3.0544074
epoch=147400,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7885768,test loss=3.0530777
epoch=147600,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7885635,test loss=3.053699
epoch=147800,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7883244,test loss=3.0552692
epoch=148000,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7883484,test loss=3.0547078
epoch=148200,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7883286,test loss=3.0550914
epoch=148400,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.788295,test loss=3.0556545
epoch=148600,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7883475,test loss=3.0539892
epoch=148800,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.78829,test loss=3.0543408
epoch=149000,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7882595,test loss=3.054684
epoch=149200,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7882724,test loss=3.0547228
epoch=149400,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7882957,test loss=3.0543952
epoch=149600,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7882178,test loss=3.0534194
epoch=149800,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.788253,test loss=3.0541775
epoch=150000,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7882495,test loss=3.0555766
epoch=150200,train accuracy=0.7271268,test accuracy=0.47201946,train loss=2.788238,test loss=3.054529
epoch=150400,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7882388,test loss=3.0549064
epoch=150600,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.788242,test loss=3.054809
epoch=150800,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7881982,test loss=3.0537488
epoch=151000,train accuracy=0.7271268,test accuracy=0.4622871,train loss=2.7882378,test loss=3.0565429
epoch=151200,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7882364,test loss=3.0545464
epoch=151400,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7882035,test loss=3.0553665
epoch=151600,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7882273,test loss=3.0554047
epoch=151800,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7882047,test loss=3.0532973
epoch=152000,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7881892,test loss=3.0547066
epoch=152200,train accuracy=0.7271268,test accuracy=0.4622871,train loss=2.7882147,test loss=3.0551865
epoch=152400,train accuracy=0.7271268,test accuracy=0.459854,train loss=2.7882164,test loss=3.055166
epoch=152600,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7881844,test loss=3.055188
epoch=152800,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.788205,test loss=3.053787
epoch=153000,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7882307,test loss=3.0550365
epoch=153200,train accuracy=0.7271268,test accuracy=0.4622871,train loss=2.7881932,test loss=3.055444
epoch=153400,train accuracy=0.7271268,test accuracy=0.4622871,train loss=2.788183,test loss=3.0543628
epoch=153600,train accuracy=0.7271268,test accuracy=0.45742092,train loss=2.788191,test loss=3.0559974
epoch=153800,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7882037,test loss=3.0538974
epoch=154000,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.788183,test loss=3.0535746
epoch=154200,train accuracy=0.7271268,test accuracy=0.4622871,train loss=2.7881868,test loss=3.0554538
epoch=154400,train accuracy=0.7271268,test accuracy=0.459854,train loss=2.7882116,test loss=3.0536985
epoch=154600,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.788197,test loss=3.054309
epoch=154800,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.788172,test loss=3.0544708
epoch=155000,train accuracy=0.7271268,test accuracy=0.4622871,train loss=2.788194,test loss=3.0538404
epoch=155200,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7881958,test loss=3.0541286
epoch=155400,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7882197,test loss=3.053475
epoch=155600,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7881696,test loss=3.0533693
epoch=155800,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7881715,test loss=3.0544894
epoch=156000,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7881842,test loss=3.0548933
epoch=156200,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.788183,test loss=3.0539382
epoch=156400,train accuracy=0.7271268,test accuracy=0.45742092,train loss=2.7881954,test loss=3.0546703
epoch=156600,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7881768,test loss=3.0548372
epoch=156800,train accuracy=0.7271268,test accuracy=0.4622871,train loss=2.7881367,test loss=3.0552
epoch=157000,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7881787,test loss=3.0537927
epoch=157200,train accuracy=0.7271268,test accuracy=0.46958637,train loss=2.7881727,test loss=3.053645
epoch=157400,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7881353,test loss=3.0535355
epoch=157600,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.788163,test loss=3.0536442
epoch=157800,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7881553,test loss=3.0532055
epoch=158000,train accuracy=0.7271268,test accuracy=0.4647202,train loss=2.7881687,test loss=3.0535793
epoch=158200,train accuracy=0.7271268,test accuracy=0.46715328,train loss=2.7881496,test loss=3.0522652